In [ ]:
# Import necessary packages
using SparseArrays
using StaticArrays
using LinearAlgebra
using IterativeSolvers
using WriteVTK
using Printf

include("Grid.jl")
include("Stokes.jl")

# Chapter 8

This code solves something "close" to Gerya's Problem 8.4. I introduce markers. The code below defines the markers, which are held in a data structure. I try to modularize all of the actions involving locating the markers, interpolating properties between the markers and the regular grid, and moving the markers.

In [ ]:
mutable struct Markers    
    x::Array{Float64,2}
    cell::Array{Int64,2}
    scalarFields::Dict
    scalars::Array{Float64,2}
    integerFields::Dict
    integers::Array{Int16,2} # note - this could be changed if larger numbers need to be stored...

    nmark::Int64
    max_mark::Int64
    
    function Markers(grid::CartesianGrid,scalarFieldNames,integerFieldNames; nmx::Integer=5,nmy::Integer=5,random::Bool=false)
        N = nmx*nmy*(grid.nx-1)*(grid.ny-1) # total number of markers
        Nmax = Int(ceil( N*1.20 ))
        mdx = grid.W/nmx/(grid.nx-1)
        mdy = grid.H/nmy/(grid.ny-1)
        
        n_fields = length(scalarFieldNames)
        scalarFields = Dict()
        ind=1
        for field in scalarFieldNames
           scalarFields[field] = ind
           ind += 1
        end
        
        n_ifields = length(integerFieldNames)
        integerFields = Dict()
        ind=1
        for field in integerFieldNames
            integerFields[field] = ind
            ind += 1
        end
        
        x = Array{Float64,2}(undef,2,Nmax)
        cell = Array{Int64,2}(undef,2,Nmax)
        
        scalars = Array{Float64,2}(undef,n_fields,Nmax)
        integers = Array{Int16,2}(undef,n_ifields,Nmax)
        
        k=1
        for i in 1:(grid.ny-1)
            for j in 1:(grid.nx-1)
                for ii in 1:nmy
                     for jj in 1:nmx
                        x[1,k] = mdx/2. + mdx*(jj-1) + mdx*nmx*(j-1) + ( random ? (rand()-0.5)*mdx : 0.0 )
                        x[2,k] = mdy/2. + mdy*(ii-1) + mdy*nmy*(i-1) + ( random ? (rand()-0.5)*mdy : 0.0 )
                        cell[1,k] = j
                        cell[2,k] = i
                        k+=1
                    end
                end
            end
        end

        new(x,cell,scalarFields,scalars,integerFields,integers,k-1,Nmax)
    end
end

In [ ]:
#
#
#     DEFINITIONS OF MARKERS AND ROUTINES FOR LOCATING THEM
#
#

function find_cell(x::Float64,gridx::Vector{Float64},nx::Int64 ; guess::Int64=nothing)
    # find the cell in the array gridx that contains the current marker
    # first, see whether the initial guess is correct.
    lower::Int64 = 1
    upper::Int64 = nx
    if guess != nothing && guess >= 1 && guess < nx
        if x >= gridx[guess] && x < gridx[guess+1]
            return guess            
        elseif guess < nx-1 && x>=gridx[guess+1] && x<gridx[guess+2]
            return guess+1
        elseif guess > 1 && x < gridx[guess] && x >= gridx[guess-1]
            return guess-1
        else
            if x>=gridx[guess+1]
                lower = guess
            else
                upper = guess+1
            end
        end
     end
    # locate cell using bisection on lower,upper
    while upper-lower > 1 
        midpoint::Int64 = lower + floor((upper-lower)/2)
        if x >= gridx[midpoint]
            lower = midpoint
        else
            upper = midpoint
        end
    end
    return lower
end

function find_cells!(markers::Markers,grid::CartesianGrid)
   for i in 1:markers.nmark
        markers.cell[1,i] = find_cell(markers.x[1,i] , grid.x, grid.nx, guess=markers.cell[1,i])
        markers.cell[2,i] = find_cell(markers.x[2,i] , grid.y, grid.ny, guess=markers.cell[2,i])
    end
end


#
#
#     ROUTINES RELATED TO MARKERS -> NODES
#
#


function marker_to_stag(m::Markers,grid::CartesianGrid,markerfields::Array,node_type::String;method="arithmetic")
    # move a list of fields (given as a list of strings in fieldnames) from markers to staggered grid
    # node type can be "basic","vx", "vy", or "center"
    stagx::Int64 = 0
    stagy::Int64 = 0
    if node_type == "basic"
        stagx=0
        stagy=0
    elseif node_type == "vx"
        stagx = 0
        stagy = -1
    elseif node_type == "vy"
        stagx = -1
        stagy = 0
    elseif node_type == "center"
        stagx = -1
        stagy = -1
    else
        error("node type unknown")
    end
    
    return marker_to_stag(m,grid,markerfields,stagx,stagy,method=method)
end

function marker_to_stag(m::Markers,grid::CartesianGrid,fieldnames::Vector{String},node_type::String;method="arithmetic")
    # move a list of fields (given as a list of strings in fieldnames) from markers to staggered grid
    # node type can be "basic","vx", "vy", or "center"
    stagx::Int64 = 0 
    stagy::Int64 = 0
    if node_type == "basic"
        stagx=0
        stagy=0
    elseif node_type == "vx"
        stagx = 0
        stagy = -1
    elseif node_type == "vy"
        stagx = -1
        stagy = 0
    elseif node_type == "center"
        stagx = -1
        stagy = -1
    else
        error("node type unknown")
    end
    
    # move a list of fields (given as a list of strings in fieldnames) from markers to cell centers.
    nfields = length(fieldnames)
    # markerfields will be indices into the 'scalars' array
    markerfields = [m.scalarFields[tmp] for tmp in fieldnames]
    return marker_to_stag(m,grid,m.scalars[markerfields,:],stagx,stagy,method=method)
end

function marker_to_stag(m::Markers,grid::CartesianGrid,markerfield::Array,stagx::Int64,stagy::Int64;method="arithmetic")
    # markerfields will be indices into the 'scalars' array
    # If stagx and stagy are zero, this function performs the same task as markers to basic nodes
    # if stagx=-1 and stagy=-1, this function performs interpolation to cell centers.
    # method can be arithmetic for the arithmetic averaging or logarithmic to take the log-average of the quantity.
    #assert(stagx == -1 || stagx == 0)
    #assert(stagy == -1 || stagy == 0)

    if method=="arithmetic"
        forward = x -> x
        inv = x -> x
    elseif method=="logarithmic"
        forward = x -> log(x)
        inv = x -> exp(x)
    elseif method=="harmonic"
        forward = x-> 1.0/x
        inv = x -> 1.0/x
    else
        error()
    end
    
    # loop over the markers    
    nfield = size(markerfield,1)
    NX::Int64 = grid.nx
    if stagx == -1 # if the grid is staggered in the x direction, pad out by one cell to include ghost nodes outside right
        NX += 1
    end
    NY::Int64 = grid.ny
    if stagy == -1 # if the grid is staggered in the y direction, pad out by one cell to include ghost nodes outside below
        NY += 1
    end
    
    weights = zeros(Float64,NY,NX)
    field = zeros(Float64,NY,NX,nfield)
    
    for i in 1:m.nmark
       # calculate weights for four surrounding grid points
         cellx::Int64 =  m.cell[1,i]
         if stagx == -1
             cellx += cellx < grid.nx && m.x[1,i] >= grid.xc[cellx+1] ? 1 : 0
         end
         celly::Int64 = m.cell[2,i]
         if stagy == -1
             celly += celly < grid.ny && m.x[2,i] >= grid.yc[celly+1] ? 1 : 0
         end
         if stagx == -1
             wx = (m.x[1,i] - grid.xc[cellx])/(grid.xc[cellx+1]-grid.xc[cellx]) # mdx/dx
         else 
            wx = (m.x[1,i] - grid.x[cellx])/(grid.x[cellx+1]-grid.x[cellx]) # mdx/dx
         end
         if stagy == -1
            wy = (m.x[2,i] - grid.yc[celly])/(grid.yc[celly+1]-grid.yc[celly])
         else
            wy = (m.x[2,i] - grid.y[celly])/(grid.y[celly+1]-grid.y[celly])
         end
         #i,j
         wt_i_j = (1.0-wx)*(1.0-wy)
         #i+1,j        
         wt_i1_j = (1.0-wx)*(wy)
         #i,j+1
         wt_i_j1 = (wx)*(1.0-wy)
         #i+1,j+1
         wt_i1_j1 = (wx)*(wy)
        
         for k in 1:nfield
             field[celly,cellx,k] += wt_i_j*forward(markerfield[k,i])
             field[celly+1,cellx,k] += wt_i1_j*forward(markerfield[k,i])
             field[celly,cellx+1,k] += wt_i_j1*forward(markerfield[k,i])
             field[celly+1,cellx+1,k] += wt_i1_j1*forward(markerfield[k,i])
        end
         weights[celly,cellx] += wt_i_j
         weights[celly+1,cellx] += wt_i1_j
         weights[celly,cellx+1] += wt_i_j1
         weights[celly+1,cellx+1] += wt_i1_j1       
    end

    return [inv.(field[:,:,k]./weights) for k in 1:nfield]
end

#
#
#     ROUTINES RELATED TO NODES -> MARKER
#
#

function basic_node_to_markers!(m::Markers,grid::CartesianGrid,field::Matrix{Float64},mfield::String)
    k = m.scalarFields[mfield]
    Threads.@threads for i in 1:m.nmark
        cellx = m.cell[1,i]
        celly = m.cell[2,i]
        wx::Float64 = (m.x[1,i] - grid.x[cellx])/(grid.x[cellx+1]-grid.x[cellx]) # mdx/dx
        wy::Float64 = (m.x[2,i] - grid.y[celly])/(grid.y[celly+1]-grid.y[celly])
        
        m.scalars[k,i] = (1.0-wx)*(1.0-wy)*field[celly,cellx] +
            + (wx)*(1.0-wy)*field[celly,cellx+1] +
            + (1.0-wx)*(wy)*field[celly+1,cellx] +
            + (wx)*(wy)*field[celly+1,cellx+1]
    end
end

function basic_node_to_markers!(m::Markers,grid::CartesianGrid,field::Matrix,mfield::Array{Float64,1})
    Threads.@threads for i in 1:m.nmark
        cellx = m.cell[1,i]
        celly = m.cell[2,i]
        wx::Float64 = (m.x[1,i] - grid.x[cellx])/(grid.x[cellx+1]-grid.x[cellx]) # mdx/dx
        wy::Float64 = (m.x[2,i] - grid.y[celly])/(grid.y[celly+1]-grid.y[celly])
        
        mfield[i] = (1.0-wx)*(1.0-wy)*field[celly,cellx] +
            + (wx)*(1.0-wy)*field[celly,cellx+1] +
            + (1.0-wx)*(wy)*field[celly+1,cellx] +
            + (wx)*(wy)*field[celly+1,cellx+1]
    end
end

function cell_center_to_markers!(m::Markers,grid::CartesianGrid,field::Matrix,mfield::Array{Float64,2})
    if size(field,1) == grid.nx+1
        cellx_max = grid.nx
    else
        cellx_max = grid.nx-1
    end
    if size(field,2) == grid.ny+1
        celly_max = grid.ny
    else
        celly_max = grid.ny-1
    end
    
    Threads.@threads for i in 1:m.nmark
        cellx = m.cell[1,i]
        celly = m.cell[2,i]
        
        cellx += cellx < cellx_max && m.x[1,i] >= grid.xc[cellx+1] ? 1 : 0
        celly::Int = m.cell[2,i]
        celly += celly < celly_max && m.x[2,i] >= grid.yc[celly+1] ? 1 : 0
        
        wx::Float64 = (m.x[1,i] - grid.xc[cellx])/(grid.xc[cellx+1]-grid.xc[cellx]) # mdx/dx
        wy::Float64 = (m.x[2,i] - grid.yc[celly])/(grid.yc[celly+1]-grid.yc[celly])
        
        mfield[1,i] = (1.0-wx)*(1.0-wy)*field[celly,cellx] +
            + (wx)*(1.0-wy)*field[celly,cellx+1] +
            + (1.0-wx)*(wy)*field[celly+1,cellx] +
            + (wx)*(wy)*field[celly+1,cellx+1]
    end
end

function cell_center_change_to_markers!(m::Markers,grid::CartesianGrid,field::Matrix,mfield::String)
    if size(field,1) == grid.nx+1
        cellx_max = grid.nx
    else
        cellx_max = grid.nx-1
    end
    if size(field,2) == grid.ny+1
        celly_max = grid.ny
    else
        celly_max = grid.ny-1
    end
    k = m.scalarFields[mfield]
    Threads.@threads for i in 1:m.nmark
        cellx = m.cell[1,i]
        celly = m.cell[2,i]
        
        cellx += cellx < cellx_max && m.x[1,i] >= grid.xc[cellx+1] ? 1 : 0
         celly::Int = m.cell[2,i]
         celly += celly < celly_max && m.x[2,i] >= grid.yc[celly+1] ? 1 : 0
        
        wx::Float64 = (m.x[1,i] - grid.xc[cellx])/(grid.xc[cellx+1]-grid.xc[cellx]) # mdx/dx
        wy::Float64 = (m.x[2,i] - grid.yc[celly])/(grid.yc[celly+1]-grid.yc[celly])
        
        m.scalars[k,i] += (1.0-wx)*(1.0-wy)*field[celly,cellx] +
            + (wx)*(1.0-wy)*field[celly,cellx+1] +
            + (1.0-wx)*(wy)*field[celly+1,cellx] +
            + (wx)*(wy)*field[celly+1,cellx+1]
    end
end

function basic_node_change_to_markers!(m::Markers,grid::CartesianGrid,field::Matrix,mfield::String)
    k = m.scalarFields[mfield]
    Threads.@threads for i in 1:m.nmark
        cellx = m.cell[1,i]
        celly = m.cell[2,i]
        wx::Float64 = (m.x[1,i] - grid.x[cellx])/(grid.x[cellx+1]-grid.x[cellx]) # mdx/dx
        wy::Float64 = (m.x[2,i] - grid.y[celly])/(grid.y[celly+1]-grid.y[celly])
        
        m.scalars[k,i] += (1.0-wx)*(1.0-wy)*field[celly,cellx] +
            + (wx)*(1.0-wy)*field[celly,cellx+1] +
            + (1.0-wx)*(wy)*field[celly+1,cellx] +
            + (wx)*(wy)*field[celly+1,cellx+1]
    end
end

function viscosity_to_cell_centers(grid::CartesianGrid,etas::Matrix{Float64})
    # compute the harmonic average of the viscosities at the nodal points
   etan = zeros(grid.ny,grid.nx)
    for i in 2:grid.ny
        for j in 2:grid.nx
            etan[i,j] = 1/( (1/etas[i-1,j-1] + 1/etas[i-1,j] + 1/etas[i,j-1] + 1/etas[i,j])/4. )
        end
    end
    return etan
end

function velocity_to_centers(grid::CartesianGrid,vx::Matrix{Float64},vy::Matrix{Float64})
    # compute vx and vy at cell centers
     vxc = zeros(grid.ny+1,grid.nx+1);
     vyc = zeros(grid.ny+1,grid.nx+1);
     # cell centers are offset in (-) direction from basic nodes.
     #               |
     # (center)     vx[i,j]
     #               |
     # ---vy[i,j]---(i,j)       
     for i in 2:grid.ny        # interior...
        for j in 2:grid.nx
            # left
            vxm = vx[i,j-1] # this will produce vx=0 along the left boundary
            vxp = vx[i,j]
            # top
            vym = vy[i-1,j] # vy=0 along the top boundary
            vyp = vy[i,j]
            vxc[i,j] = 0.5*(vxp+vxm)
            vyc[i,j] = 0.5*(vyp+vym)            
        end
    end
    # vx - top
    vxc[1,2:grid.nx] = vxc[2,2:grid.nx]
    # bottom
    vxc[grid.ny+1,2:grid.nx] = vxc[grid.ny,2:grid.nx]
    # left
    vxc[:,1] = -vxc[:,2]
    # right
    vxc[:,grid.nx+1] = - vxc[:,grid.nx]

    # vy - left
    vyc[2:grid.ny,1] = vyc[2:grid.ny,2]
    # vy - right
    vyc[2:grid.ny,grid.nx+1] = vyc[2:grid.ny,grid.nx]
    # vy - top
    vyc[1,:] = -vyc[2,:]
    # vy - bottom
    vyc[grid.ny+1,:] = -vyc[grid.ny,:]        
    
    return vxc,vyc
end

function velocity_to_basic_nodes(grid::CartesianGrid,vxc::Matrix{Float64},vyc::Matrix{Float64})
    # this gets the velocity in a format suitable for visualization.
    # NOTE - performs a transpose on the grid!!!
    vn = Array{Float64,3}(undef,2,grid.nx,grid.ny)
    for i in 1:grid.ny
        for j in 1:grid.nx
            vn[1,j,i] = 0.25*(vxc[i,j]+vxc[i+1,j]+vxc[i,j+1]+vxc[i+1,j+1])
            vn[2,j,i] = 0.25*(vyc[i,j]+vyc[i+1,j]+vyc[i,j+1]+vyc[i+1,j+1])
        end
    end
    return vn
end

function velocity_to_points(x::Matrix,cell::Matrix,grid::CartesianGrid,vxc::Matrix{Float64},vyc::Matrix{Float64};N::Int64=-1)
    # compute velocity at N points given in x (2-by-N)
    # cell should contain the cells in which the points are located (2-by-N)
    # this routine assumes that the velocity (vxc and vyc) is defined at the cell centers
    if N==-1
        N = size(x,2)
    end
    mvx = Array{Float64,1}(undef,N) # velocities at specified locations
    mvy = Array{Float64,1}(undef,N) 
    Threads.@threads for i in 1:N
        cellx::Int = x[1,i] < grid.xc[cell[1,i]+1] ? cell[1,i] : cell[1,i] + 1
        celly::Int = x[2,i] < grid.yc[cell[2,i]+1] ? cell[2,i] : cell[2,i] + 1
        mdx::Float64 = (x[1,i] - grid.xc[cellx])/(grid.xc[cellx+1]-grid.xc[cellx])
        mdy::Float64 = (x[2,i] - grid.yc[celly])/(grid.yc[celly+1]-grid.yc[celly])
        mvx[i] = (1-mdx)*(1-mdy)*vxc[celly,cellx] +
            + (mdx)*(1-mdy)*vxc[celly,cellx+1] +
            + (1-mdx)*(mdy)*vxc[celly+1,cellx] +
            + (mdx)*(mdy)*vxc[celly+1,cellx+1]
        mvy[i] = (1-mdx)*(1-mdy)*vyc[celly,cellx] +
            + (mdx)*(1-mdy)*vyc[celly,cellx+1] +
            + (1-mdx)*(mdy)*vyc[celly+1,cellx] +
            + (mdx)*(mdy)*vyc[celly+1,cellx+1]
    end    
    return mvx,mvy
end

function velocity_to_markers(m::Markers,grid::CartesianGrid,vxc::Matrix,vyc::Matrix)
    # This function expects the velocities to be defined at the cell centers. vxc and vyc should each have
    # an 'extra' column and row corresponding to the ghost degrees of freedom that are needed to interpolate
    # velocities along the bottom and left of the domain.
    mvx,mvy = velocity_to_points(m.x,m.cell,grid,vxc,vyc;N=m.nmark)
    return mvx,mvy
end

function move_markers!(markers::Markers,grid::CartesianGrid,vxc::Matrix,vyc::Matrix,dt::Float64)
    # move the markers using the 1st-order algorithm (forward Euler)
    mvx,mvy = velocity_to_markers(markers,grid,vxc,vyc)
    # determine the maximal timestep
    vxmax = maximum(abs.(mvx))
    vymax = maximum(abs.(mvy))
    Threads.@threads for i in 1:markers.nmark
        markers.x[1,i] += dt*mvx[i]
        markers.x[2,i] += dt*mvy[i]
    end    
    find_cells!(markers,grid)
    return dt
end

function move_markers_rk2!(markers::Markers,grid::CartesianGrid,vxc::Matrix{Float64},vyc::Matrix{Float64},dt::Float64)
    # move the markers using the 2nd-order Runge-Kutta algorithm.
    # compute velocities for each marker at current position
    mvx::Vector{Float64}, mvy::Vector{Float64} = velocity_to_markers(markers,grid,vxc,vyc)
    # compute marker location at xA, xB
    xB = Array{Float64,2}(undef,2,markers.nmark)
    for i in 1:markers.nmark
        xB[1,i] = markers.x[1,i] + dt/2*mvx[i]
        xB[2,i] = markers.x[2,i] + dt/2*mvy[i]
    end
    # re-locate markers, which may now be in a different cell.
    cell::Matrix{Int64} = copy(markers.cell)
    Threads.@threads for i in 1:markers.nmark
        cell[1,i] = find_cell(xB[1,i], grid.x, grid.nx, guess=cell[1,i])
        cell[2,i] = find_cell(xB[2,i], grid.y, grid.ny, guess=cell[2,i])
    end
    # compute velocity at xB
    mvx, mvy = velocity_to_points(xB,cell,grid,vxc,vyc)
    # Move the markers using the velocity at xB.
     for i in 1:markers.nmark
         markers.x[1,i] += dt*mvx[i]
         markers.x[2,i] += dt*mvy[i]
     end    
    # re-locate markers in their new cells.
    find_cells!(markers,grid)
end

In [ ]:
# convenience functions for plotting and debugging
function draw_grid(grid::CartesianGrid ; nodes::Bool=false)
   for y in grid.y
        plot([grid.x[1],grid.x[end]],[y,y],"k")
    end
    for x in grid.x
        plot([x,x],[grid.y[1],grid.y[end]],"k") 
    end
    if nodes
        for i in 1:grid.ny
            for j in 1:grid.nx
                plot(grid.y[i],grid.xc[j],"gs")
                plot(grid.yc[i],grid.x[j],"bo")
                plot(grid.yc[i],grid.xc[j],"ro")
            end
        end
    end
end

function plots(markers::Markers,grid::CartesianGrid,rho::Matrix,time)
    @views mx = markers.x[:,1:markers.nmark]
    @views mrho = markers.rho[1:markers.nmark]
    figure()
    subplot(1,2,1)
    #pcolor(grid.x,grid.y,rho)
    scatter(mx[1,:],mx[2,:],c=mrho,s=0.1)
    colorbar()
    gca().invert_yaxis()
    title(time)
    gca().set_aspect("equal")
#     draw_grid(grid)
    
    subplot(1,2,2)
    contourf(grid.x,grid.y,rho,)
    colorbar()
    gca().invert_yaxis()
    title(time)
    gca().set_aspect("equal")
end

function visualization(grid::CartesianGrid,rho::Matrix,eta::Matrix,vn::Array{Float64},pressure::Matrix,time ; filename="test.vts")
    # write the visualization output from the regular grid as a .vts file.
    vtk_grid(filename, grid.x, grid.y) do vtk
        vtk["rho"] = transpose(rho)
        vtk["viscosity"] = transpose(eta)
        # add a fake third dimension to the velocity vectors
        v3 = Array{Float64,3}(undef,3,grid.nx,grid.ny)
        v3[1:2,:,:] = vn
        v3[3,:,:] .= 0.0
        vtk["Velocity"] = v3
        vtk["pressure"] = transpose(pressure[2:end,2:end])
        vtk["TIME"] = time
    end
end

function visualization(markers::Markers,time; filename="markers.vtp")  
    p3 = Array{Float64,2}(undef,3,markers.nmark)
    p3[1:2,:] = markers.x[1:2,1:markers.nmark]
    p3[3,:] .= 0.0
      
    polys = [MeshCell(PolyData.Polys(),i:i) for i in 1:markers.nmark]
    vtk_grid(filename,p3,polys) do vtk    
        for key in keys(markers.scalarFields)
            vtk[key] = markers.scalars[markers.scalarFields[key],1:markers.nmark]
        end
       vtk["TIME"] = time
    end
end

In [ ]:
function initial_conditions!(markers::Markers)
    for i in 1:markers.nmark
        mx = markers.x[1,i]
        my = markers.x[2,i]
        mr = ((mx-2.5e5)^2 + (my-2.5e5)^2)^0.5
        # my initial misunderstanding of the problem setup:
        # markers.rho[i] = mx >= 200e3 && mx <= 300e3 ? 3200. : 3300.
        rho = markers.scalarFields["rho"]
        eta = markers.scalarFields["eta"]
        mat = markers.integerFields["mat"]
        markers.scalars[rho,i] = mr < 1.0e5 ? 3200. : 3300.
        markers.scalars[eta,i] = mr < 1.0e5 ? 1e20 : 1e21 
        markers.integers[mat,i] = mr < 1.0e5 ? 1 : 2
    end
end

# Test marker-related routines
The cell below test the marker-related routines.
1. Create markers and distribute uniformly in the domain.
2. Apply the initial conditions to the markers.
3. Calculate density at the basic nodes from the markers.
4. Interpolate the density back to the markers from the basic nodes.

In [ ]:
# Diagnostic tests on marker routines:
# Set up a grid and some markers
nx = 8
ny = 7
W = 5e5
H = 5e5
gy = -10.0
gx =  0.0
markx = 5
marky = 4
grid = CartesianGrid(W,H,nx,ny)
markers = Markers(grid,["rho","eta"],["mat"] ; nmx=markx,nmy=marky,random=false)

mx = [markers.x[1,i] for i in 1:markers.nmark]
my = [markers.x[2,i] for i in 1:markers.nmark]
mcx = [markers.cell[1,i] for i in 1:markers.nmark]
mcy = [markers.cell[2,i] for i in 1:markers.nmark]

using PyPlot
figure()
scatter(mx,my,c=mcx)
draw_grid(grid)
colorbar()
show()

initial_conditions!(markers)

figure()
mrho = markers.scalars[markers.scalarFields["rho"],1:markers.nmark]
scatter(mx,my,c=mrho)
draw_grid(grid)
title("Initial density")
colorbar()
show()


rho_node,eta_node = marker_to_stag(markers,grid,["rho","eta"],"basic")
print(rho_node)
figure()
pcolormesh(grid.x,grid.y,rho_node)
mrho = markers.scalars[markers.scalarFields["rho"],1:markers.nmark]
scatter(mx,my,c=mrho,s=0.1)
draw_grid(grid)
title("density from markers")
colorbar()
show()

basic_node_to_markers!(markers,grid,rho_node,"rho")

figure()
mrho = markers.scalars[markers.scalarFields["rho"],1:markers.nmark]
scatter(mx,my,c=mrho)
draw_grid(grid)
title("Rho post-interpolation")
colorbar()
show()

# visualization routines

visualization(markers,0.0)

In [ ]:
# Set up the grid
nx = 51
ny = 52
W = 5e5
H = 5e5
gy = 10.0
gx =  0.0
markx = 5
marky = 5
seconds_in_year = 3.15e7
plot_interval = 1e5*seconds_in_year # plot interval in seconds
end_time = 30e6*seconds_in_year # end time in seconds
dtmax = plot_interval
bc = BoundaryConditions(0,0,0,0) # currently does nothing but is required argument to stokes solver.

grid = CartesianGrid(W,H,nx,ny)
println("Creating Markers...")
@time markers = Markers(grid,["rho","eta"],["mat"] ; nmx=markx,nmy=marky,random=false)

# Initial conditions
initial_conditions!(markers)

time = 0.0
iout = 0
last_plot = 0.0

while time <= end_time
    # interpolate rho and eta from markers to basic nodes


    rho_new, eta_s_new = marker_to_stag(markers,grid,["rho","eta"],"basic") 
    rho_vx_new, = marker_to_stag(markers,grid,["rho",],"vx");
    rho_vy_new, = marker_to_stag(markers,grid,["rho",],"vy");
    println(time)
    if time > 0.0
        nanind = findall(isnan.(rho_new))
        rho_new[nanind] = rho[nanind]
        eta_s_new[nanind] = eta_s[nanind]
        nanind = findall(isnan.(rho_vx_new))
        rho_vx_new[nanind] = rho_vx[nanind]
        nanind = findall(isnan.(rho_vy_new))
        rho_vy_new[nanind] = rho_vy[nanind]
        
    end
    
    global rho   = copy(rho_new)
    global eta_s = copy(eta_s_new)    
    global rho_vx = copy(rho_vx_new)
    global rho_vy = copy(rho_vy_new)
    
    # compute eta at cell centers
    eta_n = viscosity_to_cell_centers(grid,eta_s)
   
    L,R = form_stokes(grid,eta_s,eta_n,rho_vx,rho_vy,bc,gx,gy)
    solution = L\R
    vx,vy,P = unpack(solution,grid;ghost=true)
    vxc,vyc = velocity_to_centers(grid,vx,vy)
    
    if time == 0.0 || time - last_plot >= plot_interval
        last_plot = time 
        name = @sprintf("output_chapter8/viz.%04d.vtr",iout)
        vn = velocity_to_basic_nodes(grid,vxc,vyc)
        visualization(grid,rho,eta_s,vn,P,time/seconds_in_year;filename=name)
        name = @sprintf("output_chapter8/markers.%04d.vtp",iout)
        visualization(markers,time/seconds_in_year;filename=name)
        iout += 1
    end
    
    dt = compute_timestep(grid,vxc,vyc;dtmax=dtmax)
    move_markers_rk2!(markers,grid,vxc,vyc,dt)
    time += dt
end

In [ ]:
# tests for a single step at high resolution.
nx = 201
ny = 202
W = 5e5
H = 5e5
gy = -10.0
gx =  0.0
markx = 5
marky = 5
plot_interval = 10
dtmax = 1e6*3.15e7

grid = CartesianGrid(W,H,nx,ny)
println("Creating Markers...")
@time markers = Markers(grid,["rho","eta"],["mat"] ; nmx=markx,nmy=marky,random=false)
# Initial conditions
initial_conditions!(markers)
println("markers to basic nodes...")
@time rho,eta_s = marker_to_stag(markers,grid,["rho","eta"],"basic")
@time rho_vx, = marker_to_stag(markers,grid,["rho",],"vx");
@time rho_vy, = marker_to_stag(markers,grid,["rho",],"vy");
#eta_s = 1e21 .* ones(grid.ny,grid.nx)
eta_n = 1e21 .* ones(grid.ny,grid.nx)

# rho,eta_s,eta_n = initial_conditions(grid)
bc = BoundaryConditions(0,0,0,0)
println("forming stokes...")
@time L,R = form_stokes(grid,eta_s,eta_n,rho_vx,rho_vy,bc,gx,gy)
println("solving stokes...")
@time solution = L\R
println("unpacking...")
@time vx,vy,P = unpack(solution,grid;ghost=true)
println("cell center velocities...")
vxc,vyc = velocity_to_centers(grid,vx,vy)
println("Computing timestep...")
@time dt=compute_timestep(grid,vxc,vyc)
println("advecting markers...")
@time move_markers_rk2!(markers,grid,vxc,vyc,dt)
println("visualization...")
vn = velocity_to_basic_nodes(grid,vxc,vyc)
@time visualization(grid,rho,eta_s,vn,P,0.0,filename="test.vtr")

In [ ]:
mat, = marker_to_stag(markers,grid,markers.integers[[markers.integerFields["mat"]],:],"basic")
figure()
pcolor(mat)
show()
